In [1]:
!pip install google -i http://pypi.douban.com/simple  --trusted-host pypi.douban.com

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install protobuf -i http://pypi.douban.com/simple  --trusted-host pypi.douban.com

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [72]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# convert tmall session data to our protobuf data

import google.protobuf
import google.protobuf.text_format
import chat_pb2
import json
import re

# line: 2018-05-31
def extract_date(line):
    date_ptn = r'(\d{4}-\d{2}-\d{2})'
    m = re.match(date_ptn, line)
    date = m.group(1)
    return date

# line: a979289485 (11:43:39)
def extract_speaker(line): 
    speaker_ptn = r'(.*)\((\d{2}:\d{2}:\d{2})\)'
    m = re.match(speaker_ptn, line)
    return m.group(1).strip()

# line: a979289485 (11:43:39)
def extract_time(line): 
    speaker_ptn = r'(.*)\((\d{2}:\d{2}:\d{2})\)'
    m = re.match(speaker_ptn, line)
    return m.group(2).strip()

# print extract_speaker('a979289485 (11:43:39)')

# convert taobao json to chat pb
def convert(inFile, outFile):
    taobao_chatbot_data_file = inFile

    with open(taobao_chatbot_data_file, 'r') as fin:
        json_data = json.load(fin)

    chat = chat_pb2.Chat()
    for session_data in json_data['tmall_kefu_data']:
        turn_data_index = 0
        date = ''
        session = chat.session.add()
        turn = None
        for turn_data in session_data['data']:
            
            # 第0个数，是发生日期
            # 第奇数个数，是speaker
            # 第偶数个数，是content
            content = turn_data['content']
            if turn_data_index == 0:
                date = extract_date(content)
                turn_data_index = turn_data_index + 1
                continue
                
            if turn_data_index % 2 == 1:  # 奇数
                turn = session.turn.add()
                speaker = extract_speaker(content)
                time = extract_time(content)
                turn.time = date + ' ' + time
                turn.speaker = speaker
            elif turn_data_index % 2 == 0:  # 偶数
                query = ''
                if content:
                    query = content
                face_title = turn_data['face_title']
                face_url = turn_data['face_url']
                if face_title and face_url:
                    query = '[img_url=' + face_url + '][img_title =' + face_title + ']' + query
                turn.query.append(query)
            turn_data_index = turn_data_index + 1
        #print 'session:'
        #print session
        # print session.SerializeToString().encode('utf-8')
        # print text_format.MessageToString(session, as_utf8=True)
        # break
    # print 'Fin'
    #text_format.MessageToString(sessions[0], as_utf8=True)
    print 'Fin'
    with open(outFile, "w") as fout:
        fout.write(chat.SerializeToString())
    
convert( u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.json', u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb')

Fin


In [108]:
#!/usr/bin/python
#-*- coding:utf-8 -*-

# annoymize the pb query which has user name, user address, and user phones

import google.protobuf
import chat_pb2
import re

cn_mobile = [134,135,136,137,138,139,150,151,152,157,158,159,182,183,184,187,188,147,178,1705]
cn_union = [130,131,132,155,156,185,186,145,176,1709]
cn_telecom = [133,153,180,181,189,177,1700]
def detect_cellphone(line):
    ptn1 = r'.*(\d{11}).*'
    m = re.match(ptn1, line)
    if not m:
        return False
    phone = m.group(1)
    ptn2 = '^(' + '|'.join([str(phone_prefix) for phone_prefix in cn_mobile + cn_union + cn_telecom]) + ')'
    if not re.match(ptn2, phone):
        return False
    return True

#print detect_cellphone('~~17633520709')

def anonymize_query(query):
    if detect_cellphone(query):
        print u'anonymize query from ' + query + u' to <联系方式>.'
        return u'<联系方式>'
    else:
        return query
    
def anonymize(inFile, outFile):
    chat = chat_pb2.Chat()
    with open(inFile, 'r') as fin:
        chat.ParseFromString(fin.read())
    
    speakers = set()
    for session in chat.session:
        for turn in session.turn:
            anonymized_query = [anonymize_query(query) for query in turn.query]
            turn.ClearField('query')
            turn.query.extend(anonymized_query)
    
    with open(outFile, "w") as fout:
        fout.write(chat.SerializeToString())
        
anonymize(u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb', \
          u'/Users/qli/Documents/墨百意/问问客服数据/tmall_chatbot_data.pb.anonymized')

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，上限是20元。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 南宁市青秀区桂雅路11号麒麟大厦b1905 18587865325李辉 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，上限是20元。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 福建省泉州市石狮市湖东水果批发市场玉池东路16号 13276099580 闻康 to <联系方式>.
anonymize query from 湖南省岳阳市岳阳楼区望岳路何阳组285号 周军华13973052805 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在

anonymize query from 梁国振  18307713720  广西南宁市江南区吴圩镇明阳石油基地 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，上限是20元。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 我的电话18936361677  张子洋 to <联系方式>.
anonymize query from 收货人：ticwatch售后 地址：北京市海淀区北四环西路67号（中关村国际创新大厦）502室   13691408229亲，请您在商品发回前。根据以下注意事项做好相关检查工作哦~一、商品均保证不影响二次销售（无划痕、表带无褶皱痕迹、无人为损坏、无硬伤等）赠品全部寄回二、拒签到付和平邮 如由于产品质量问题退货，请您先垫付运费。后续问题处理后，属于产品本身问题。我们将给您退还您退回所承担的费用（上限20元）三、请您在退回商品中附加纸条，填写您购买商品的相关信息否则我们无法找到您的订单（购买平台、购物ID、购买人姓名、电话、以及相关产品问题描述）四、手表 3G版本 激活3G或NFC功能不能退货， 入网许可证蓝标不可以撕坏五：如果收到纸质发票请一同寄回由于以上问题所产生的拒签问题，我们不承担任何风险责任，建议使用以下快递公司效率更快哦~申通 韵达 百世汇通 天天 中通 顺丰  请您再联系我们之后的3天内寄出货物 否则过期无法给您处理  亲青岛正在开会，如果您是青岛的无法寄处快递 请您联系在线客服告知，然后会议结束之后1天内寄出货物，如超期无法退货 to <联系方式>.
anonymize query from 

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。付费维修都会将您贴上去的钢化膜揭掉~运费需要您自己承担。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 感谢您的支持 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 2018-06-12 17:25:00 快件交给王涛，正在派送途中（联系电话：15615629292）    亲亲，您拨打下这个快递员的电话看下 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。付费维修都会将您贴上去的钢化膜揭掉~运费需要您自己承担。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 感谢您的支持 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区北四环西路67号（中关村国际创新大厦）502室 注意：请您务必将手表整套（除赠品外）寄回，具体包括：手表外包装盒，手表，充电器整套，说明书，其余小包装盒损坏不会影响您换新~请您务必在寄回包裹中附加纸条，写明：姓名，电话，手表款式以及具体问题。否则售后工程师将无法处理您的订单，只能搁置处理或者拒签处理。拒收平邮和到付件。运费需要您先补垫，收到换新后的手表，检查无误，请联系我们给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 18637017960李国豪 to <联系方式>.
anonymize query from 李浩  18638590358 to <联系方式>.
anonymize query from 13215042500  卢志辉 to <联系方式>.
anonymize query from 收货人：ticwatch售后 地址：北京市海淀区北四环西路67号（中关村国际创新大厦）502室   13691408229亲，请您在

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 收货人：ticwatch售后 地址：北京市海淀区北四环西路67号（中关村国际创新大厦）502室   13691408229亲，请您在商品发回前。根据以下注意事项做好相关检查工作哦~一、商品均保证不影响二次销售（无划痕、表带无褶皱痕迹、无人为损坏、无硬伤等）赠品全部寄回二、拒签到付和平邮 如由于产品质量问题退货，请您先垫付运费。后续问题处理后，属于产品本身问题。我们将给您退还您退回所承担的费用（各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元 西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费））三、请您在退回商品中附加纸条，填写您购买商品的相关信息否则我们无法找到您的订单（购买平台、购物ID、购买人姓名、电话、以及相关产品问题描述）四、手表 3G版本 激活3G或NFC功能不能退货， 入网许可证蓝标不可以撕坏五：如果收到纸质发票请一同寄回由于以上问题所产生的拒签问题，我们不承担任何风险责任，

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。付费维修都会将您贴上去的钢化膜揭掉~运费需要您自己承担。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 感谢您的支持 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区北四环西路67号（中关村国际创新大厦）502室 注意：请您务必将手表整套（除赠品外）寄回，具体包括：手表外包装盒，手表，充电器整套，说明书，其余小包装盒损坏不会影响您换新~请您务必在寄回包裹中附加纸条，写明：姓名，电话，手表款式以及具体问题。否则售后工程师将无法处理您的订单，只能搁置处理或者拒签处理。拒收平邮和到付件。运费需要您先补垫，收到换新后的手表，检查无误，请联系我们给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 系统提示: 用户 zxf13794809901 已由 ticwatch旗舰店:cs7 转交给ticwatch旗舰店:智能小表哥，请尽快回复! to <联系方式>.
anonymize query from 215553079610 to <联系方式>.
anonymize query from 您如果还需要，您可以拨打咱们负责人电话： 13260114771 to <联系方式>.
anonymize query fr

anonymize query from 地址，贵州省  六盘水市  钟山区  荷城街道钟山大道 帝都新城一期 十二号楼  姓名 许愿  电话18768696621 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 便签姓名和电话。王瑞 15902222608 to <联系方式>.
anonymize query from 系统提示: 用户 15122232170li 已由 ticwatch旗舰店:cs11 转交给ticwatch旗舰店:智能小表哥，请尽快回复! to <联系方式>.
anonymize query from 15122232170li 撤回了一条消息 to <联系方式>.
anonymize query from 买家姓名：李华鑫联系方式：18233654991 收货地址：河北省 沧州市 盐山县 盐山镇红庙大街阜德小区买家邮编：061300 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。付费维修都会将您贴上去的钢化膜揭掉~运费需要您自己承担。建议使用：申通 韵达 百世汇通 天天 中通寄回哦~ 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 感谢您的支持 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海  地区 退运费金额12元西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额10元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理 to <联系方式>.
anonymize query from 王云凯  18274587421 to <联系方式>.
anonymize query from 周雄13695852858 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，

anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。质保维修都会将您贴上去的钢化膜揭掉~运费需要您先补垫，收到修好后的手表，检查无误，请联系我们，质量问题可以给您补运费，各地区上限金额：  内蒙古， 甘肃， 宁夏， 青海 、西藏 ， 新疆    地区 退运费金额15元除以上地区外，其他地区退运费金额12元快递可接收：圆通 ，申通，中通，韵达，百事汇通不接受：平邮ems，顺丰到付（如需使用，超出运费金额自行承担。以及不接受快递费用额外的增值费用，比如保价费）。 感谢亲的配合~祝您生活愉快~  请您再联系我们之后的3天内寄回货物 否则过期无法给您处理亲如果您是质量问题报销运费，请您保留好了快递底单上有快递金额的快递底单，提供给我们 to <联系方式>.
anonymize query from 内蒙古呼和浩特市赛罕区巨海城八区 陈恺峰 15849373816 to <联系方式>.
anonymize query from 有什么事发信息都这个手机号码13662753220手机不方便接电话 to <联系方式>.
anonymize query from 广州市番禺区钟村镇雄峰城A区南大街39号德国诗珑漆      黄亿13422034983 to <联系方式>.
anonymize query from 彭乐乐  17671456901 to <联系方式>.
anonymize query from 收货人：ticwatch售后 电话：13691408229地址：北京市海淀区中关村中关村国际创新大厦5层502 室13691408229注意：请务必在寄回包裹中附加纸条，写明：姓名，电话，并且把便签的电话和姓名提供给在线客服，还有手表款式以及具体问题?。寄回物品包括：手表和充电器！否则售后工程师将无法处理您的订单，拒收平邮和到付，只能搁置处理或者拒签处理。付费维修都会将您贴上去的钢化膜揭掉~运费需要您自己承担。建议使用：申通 韵达 百